In [2]:
import cv2
import numpy as np 
import pandas as pd 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input

In [3]:
df_labels = pd.read_csv("labels.csv")
train_file = 'train/'
test_file = 'test/'

In [4]:
df_labels

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
...,...,...
10217,ffd25009d635cfd16e793503ac5edef0,borzoi
10218,ffd3f636f7f379c51ba3648a9ff8254f,dandie_dinmont
10219,ffe2ca6c940cddfee68fa3cc6c63213f,airedale
10220,ffe5f6d8e2bff356e9482a80a6e29aac,miniature_pinscher


In [5]:
print("Total number of unique Dog Breeds :",len(df_labels.breed.unique()))

Total number of unique Dog Breeds : 120


In [6]:
num_breeds = 60
im_size = 224
batch_size = 64
encoder = LabelEncoder()

In [7]:
breed_dict = list(df_labels['breed'].value_counts().keys()) 
new_list = sorted(breed_dict,reverse=True)[:num_breeds*2+1:2]
df_labels = df_labels.query('breed in @new_list')

In [8]:
new_list

['yorkshire_terrier',
 'whippet',
 'welsh_springer_spaniel',
 'walker_hound',
 'toy_terrier',
 'tibetan_terrier',
 'sussex_spaniel',
 'standard_poodle',
 'soft-coated_wheaten_terrier',
 'siberian_husky',
 'shetland_sheepdog',
 'scottish_deerhound',
 'schipperke',
 'saluki',
 'rottweiler',
 'redbone',
 'pomeranian',
 'pekinese',
 'otterhound',
 'norwich_terrier',
 'norfolk_terrier',
 'miniature_schnauzer',
 'miniature_pinscher',
 'maltese_dog',
 'malamute',
 'leonberg',
 'labrador_retriever',
 'komondor',
 'kelpie',
 'japanese_spaniel',
 'irish_wolfhound',
 'irish_terrier',
 'ibizan_hound',
 'greater_swiss_mountain_dog',
 'great_dane',
 'golden_retriever',
 'german_short-haired_pointer',
 'french_bulldog',
 'eskimo_dog',
 'english_springer',
 'english_foxhound',
 'dingo',
 'dandie_dinmont',
 'collie',
 'clumber',
 'chihuahua',
 'cardigan',
 'bull_mastiff',
 'briard',
 'boxer',
 'boston_bull',
 'border_terrier',
 'bluetick',
 'blenheim_spaniel',
 'bernese_mountain_dog',
 'beagle',
 'base

In [13]:
df_labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [14]:
df_labels['img_file'] = df_labels['id'].apply(lambda x: x + ".jpg")

C:\Users\mrmel\AppData\Local\Temp\ipykernel_1564\1967024964.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_labels['img_file'] = df_labels['id'].apply(lambda x: x + ".jpg")


In [15]:
df_labels

,id,breed,img_file
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull,000bec180eb18c7604dcecc8fe0dba07.jpg
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo,001513dfcb2ffafc82cccf4d8bbaba97.jpg
2,001cdf01b096e06d78e9e5112d419397,pekinese,001cdf01b096e06d78e9e5112d419397.jpg
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick,00214f311d5d2247d5dfe4fe24b2303d.jpg
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever,0021f9ceb3235effd7fcde7f7538ed62.jpg
...,...,...,...
10214,ffcb610e811817766085054616551f9c,briard,ffcb610e811817766085054616551f9c.jpg
10215,ffcde16e7da0872c357fbc7e2168c05f,airedale,ffcde16e7da0872c357fbc7e2168c05f.jpg
10218,ffd3f636f7f379c51ba3648a9ff8254f,dandie_dinmont,ffd3f636f7f379c51ba3648a9ff8254f.jpg
10219,ffe2ca6c940cddfee68fa3cc6c63213f,airedale,ffe2ca6c940cddfee68fa3cc6c63213f.jpg


In [16]:
train_x = np.zeros((len(df_labels), im_size, im_size, 3), dtype='float32')
for i, img_id in enumerate(df_labels['img_file']):
  img = cv2.resize(cv2.imread(train_file+img_id,cv2.IMREAD_COLOR),((im_size,im_size)))
  img_array = preprocess_input(np.expand_dims(np.array(img[...,::-1].astype(np.float32)).copy(), axis=0))
  train_x[i] = img_array

In [14]:
train_y = encoder.fit_transform(df_labels["breed"].values)

In [15]:
x_train, x_test, y_train, y_test = train_test_split(train_x,train_y,test_size=0.2,random_state=42)

In [16]:
train_datagen = ImageDataGenerator(rotation_range=45,width_shift_range=0.2,height_shift_range=0.2,
                                   shear_range=0.2,zoom_range=0.25,horizontal_flip=True,fill_mode='nearest')
train_generator = train_datagen.flow(x_train,y_train,batch_size=batch_size)
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow(x_test,y_test,batch_size=batch_size)

In [17]:
resnet = ResNet50V2(input_shape = [im_size,im_size,3], weights='imagenet', include_top=False)
for layer in resnet.layers:
    layer.trainable = False
x = resnet.output
x = BatchNormalization()(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)

94668760/94668760 [==============================] - 26s 0us/step


In [1]:
predictions = Dense(num_breeds, activation='softmax')(x)
model = Model(inputs=resnet.input, outputs=predictions)

NameError: name 'Dense' is not defined

In [19]:
epochs = 20
learning_rate = 1e-3
optimizer = RMSprop(learning_rate=learning_rate,rho=0.9)
model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=["accuracy"])
hist = model.fit(train_generator,steps_per_epoch= x_train.shape[0] // batch_size,epochs= epochs,
                 validation_data= test_generator,validation_steps= x_test.shape[0] // batch_size)
model.save("dogbreedmodel.h5")

Epoch 1/20
64/64 [==============================] - 565s 9s/step - loss: 2.2446 - accuracy: 0.4325 - val_loss: 0.8102 - val_accuracy: 0.7490
Epoch 2/20
64/64 [==============================] - 581s 9s/step - loss: 1.2624 - accuracy: 0.6305 - val_loss: 0.7000 - val_accuracy: 0.7773
Epoch 3/20
64/64 [==============================] - 565s 9s/step - loss: 1.0925 - accuracy: 0.6776 - val_loss: 0.6618 - val_accuracy: 0.7910
Epoch 4/20
64/64 [==============================] - 562s 9s/step - loss: 0.9934 - accuracy: 0.7041 - val_loss: 0.6655 - val_accuracy: 0.8047
Epoch 5/20
64/64 [==============================] - 559s 9s/step - loss: 0.9355 - accuracy: 0.7196 - val_loss: 0.6551 - val_accuracy: 0.7988
Epoch 6/20
64/64 [==============================] - 581s 9s/step - loss: 0.9148 - accuracy: 0.7260 - val_loss: 0.6655 - val_accuracy: 0.8047
Epoch 7/20
64/64 [==============================] - 553s 9s/step - loss: 0.8981 - accuracy: 0.7267 - val_loss: 0.6657 - val_accuracy: 0.7969
Epoch 8/20
64

C:\Users\mrmel\anaconda3\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
